In [ ]:
from mnemonic import Mnemonic
from bip_utils import Bip39SeedGenerator, Bip44, Bip44Coins, Bip44Changes, Bip44Levels
from cryptography.hazmat.primitives.asymmetric import ec
from cryptography.hazmat.primitives import serialization
from cryptography.hazmat.backends import default_backend

def generate_keys_from_mnemonic(mnemonic_phrase):
    # 验证助记词
    mnemo = Mnemonic("english")
    if not mnemo.check(mnemonic_phrase):
        raise ValueError("Invalid mnemonic phrase")

    # 从助记词生成种子
    seed = Bip39SeedGenerator(mnemonic_phrase).Generate()

    # 使用BIP-44路径生成主私钥
    bip_obj = Bip44.FromSeed(seed, Bip44Coins.ETHEREUM)
    bip_obj_address = bip_obj.Purpose().Coin().Account(0).Change(Bip44Changes.CHAIN_EXT).AddressIndex(0)

    # 获取主私钥
    master_private_key = bip_obj_address.PrivateKey().Raw().ToBytes()

    # 从主私钥生成ECC密钥对
    private_key = ec.derive_private_key(int.from_bytes(master_private_key, byteorder="big"), ec.SECP256R1(), default_backend())

    # 将私钥保存到文件
    private_pem = private_key.private_bytes(
        encoding=serialization.Encoding.PEM,
        format=serialization.PrivateFormat.PKCS8,
        encryption_algorithm=serialization.NoEncryption()
    )
    with open('private_key_from_mnemonic.pem', 'wb') as f:
        f.write(private_pem)

    # 将公钥保存到文件
    public_pem = private_key.public_key().public_bytes(
        encoding=serialization.Encoding.PEM,
        format=serialization.PublicFormat.SubjectPublicKeyInfo
    )
    with open('public_key_from_mnemonic.pem', 'wb') as f:
        f.write(public_pem)

    print("Keys generated from mnemonic and saved successfully.")

if __name__ == '__main__':
    mnemonic_phrase = "your twelve or twenty-four word mnemonic phrase here"
    generate_keys_from_mnemonic(mnemonic_phrase)

In [ ]:
from eth_account import Account
import os

def generate_keys_from_mnemonic(mnemonic_phrase):
    # 从助记词恢复账户
    account = Account.from_mnemonic(mnemonic_phrase)

    # 获取私钥
    private_key = account.key.hex()

    # 获取公钥
    public_key = account._key.public_key.to_hex()

    # 将私钥保存到文件
    with open('private_key_from_mnemonic.pem', 'w') as f:
        f.write(private_key)

    # 将公钥保存到文件
    with open('public_key_from_mnemonic.pem', 'w') as f:
        f.write(public_key)

    print("Keys generated from mnemonic and saved successfully.")

if __name__ == '__main__':
    mnemonic_phrase = "your twelve or twenty-four word mnemonic phrase here"
    generate_keys_from_mnemonic(mnemonic_phrase)

In [ ]:
from cryptography.hazmat.primitives.asymmetric import ec
from cryptography.hazmat.primitives import serialization
from cryptography.hazmat.backends import default_backend

def convert_to_pem(private_key_hex, public_key_hex):
    # 将私钥从十六进制字符串转换为字节
    private_key_bytes = bytes.fromhex(private_key_hex)
    
    # 从字节创建私钥对象
    private_key = ec.derive_private_key(int.from_bytes(private_key_bytes, byteorder="big"), ec.SECP256R1(), default_backend())
    
    # 将私钥保存为PEM格式
    private_pem = private_key.private_bytes(
        encoding=serialization.Encoding.PEM,
        format=serialization.PrivateFormat.PKCS8,
        encryption_algorithm=serialization.NoEncryption()
    )
    with open('private_key.pem', 'wb') as f:
        f.write(private_pem)
    
    # 从私钥对象获取公钥对象
    public_key = private_key.public_key()
    
    # 将公钥保存为PEM格式
    public_pem = public_key.public_bytes(
        encoding=serialization.Encoding.PEM,
        format=serialization.PublicFormat.SubjectPublicKeyInfo
    )
    with open('public_key.pem', 'wb') as f:
        f.write(public_pem)
    
    print("Keys converted to PEM format and saved successfully.")

if __name__ == '__main__':
    private_key_hex = "your_private_key_in_hex_format_here"
    public_key_hex = "your_public_key_in_hex_format_here"
    convert_to_pem(private_key_hex, public_key_hex)

In [ ]:
from eth_account import Account
import hashlib

def sign_message(private_key_hex, message):
    # 从私钥创建账户对象
    account = Account.from_key(private_key_hex)

    # 计算消息的哈希值
    message_hash = hashlib.sha256(message.encode()).digest()

    # 对消息哈希进行签名
    signature = account.signHash(message_hash)

    return signature.signature.hex()

if __name__ == '__main__':
    private_key_hex = "your_private_key_in_hex_format_here"
    message = "Hello, world!"

    signature = sign_message(private_key_hex, message)
    print(f"Signature: {signature}")

In [ ]:
from eth_account import Account
import hashlib

def verify_signature(public_key_hex, message, signature_hex):
    # 计算消息的哈希值
    message_hash = hashlib.sha256(message.encode()).digest()

    # 从公钥创建账户对象
    public_key_bytes = bytes.fromhex(public_key_hex)
    account = Account.from_key(Account._keys.PrivateKey(public_key_bytes))

    # 验证签名
    try:
        recovered_address = Account.recoverHash(message_hash, signature=bytes.fromhex(signature_hex))
        expected_address = account.address
        return recovered_address == expected_address
    except Exception as e:
        print(f"Verification failed: {e}")
        return False

if __name__ == '__main__':
    private_key_hex = "your_private_key_in_hex_format_here"
    message = "Hello, world!"
    signature = sign_message(private_key_hex, message)

    # 从私钥推导公钥
    account = Account.from_key(private_key_hex)
    public_key_hex = account._key.public_key.to_hex()

    is_valid = verify_signature(public_key_hex, message, signature)
    print(f"Signature valid: {is_valid}")